In [0]:
%sql
use catalog pricing_analytics;
insert into silver.daily_pricing_silver
select 
to_date(DATE_OF_PRICING, "dd/mm/yyyy") ,
cast(ROW_ID as bigint), 	
STATE_NAME,
MARKET_NAME,
PRODUCTGROUP_NAME,
PRODUCT_NAME,
VARIETY,
ORIGIN,
try_cast( ARRIVAL_IN_TONNES as decimal(18,2)) ,
try_cast( MINIMUM_PRICE as	decimal(36,2)),
try_cast( MAXIMUM_PRICE	as decimal(36,2)),
try_cast( MODAL_PRICE	as decimal(36,2)),
source_file_load_date, 
current_timestamp(),
current_timestamp()
from pricing_analytics.bronze.daily_pricing 
where source_file_load_date >
(select max(PROCESSED_TABLE_DATE_TIME) 
from  pricing_analytics.processrunlogs.deltalakehouse_process_runs 
where process_name = 'daily_pricing_silver' and PROCESS_STATUS= 'Completed') ; 



In [0]:
%sql
insert into pricing_analytics.processrunlogs.deltalakehouse_process_runs (PROCESS_NAME, PROCESS_STATUS, PROCESSED_TABLE_DATE_TIME)
select 'daily_pricing_silver', 'Completed', nvl(max(source_file_load_date), "1900-01-01" )  from pricing_analytics.silver.daily_pricing_silver